# Trade Tracking Simulation

 -  Time Point : 1 minute

#### 01. 매도 조건

 1. 목표 수익률 (7%) 달성시 매도 ***(구현 완료)***
 
 
 2. 손절 요건 미충족시 그날 15:20 종가로 매매 ***(구현 완료)***

#### 02. 손절 조건 (아래 요건 중 1개만 달성해도 매도)

 1. 매입가대비 5% 이상 하락한 경우 ***(구현 완료)***


 2. 직전 Time Point 기준 종가가 2% 이상 하락 현상이 나타난 경우 ***(구현 완료)***
  
 
 3. 매입가 대비 15:00 의 종가가 오차범위(-0.5 ~ 0.5%) 이내일 때 조기 매도 ***(구현 완료)*** 

#### 03. 종목 셋팅

In [1]:
date = '20200323' 

In [2]:
code = '141020'

#### 04. 시뮬레이션

In [10]:
# 경고 무시
import warnings
warnings.filterwarnings(action='ignore') 

# Hyper Parameter
tr_name = 'TR_SCHART'
term = '1'
start_date = date
end_date = date
Lookup = '9999' 
        
# url
url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=' + term + '&2=5' +\
      '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup

# url open
from urllib.request import urlopen
url_page = urlopen(url)

url

ImportError: No module named request

In [3]:
# 경고 무시
import warnings
warnings.filterwarnings(action='ignore') 

# Hyper Parameter
tr_name = 'TR_SCHART'
term = '1'
start_date = date
end_date = date
Lookup = '9999' 
        
# url
url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=' + term + '&2=5' +\
      '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup

# url open
from urllib.request import urlopen
url_page = urlopen(url)

# json 파일로 받아오기
import json
url_data = json.loads(url_page.read())

# json을 데이터프레임으로 바꾸기
import pandas as pd
data = pd.DataFrame(url_data)
    
# Data Column명 변경
data.columns = ['date','time','open','high','low','close','price_ccr','volume_ccr',
                'rock','volume','volume_price']
data = data[['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'volume_price']]
    
# 순서 뒤집기
data = data[::-1]
data['date_time'] = data['date'] + '_' + data['time']
data = data.set_index('date_time')
    
# 필요한 변수만 추출
data = data[['open', 'high', 'low', 'close', 'volume']]

# 변수 속성 정수로 바꾸기
for column in data.columns : 
    data[column] = data[column].apply(lambda x : int(x))

# 진행상태 표시
data['state'] = ''

# Simulation
simulation_df = pd.DataFrame()

for idx in data.index : 
        
    # 시가에 매수
    if idx == date + '_0905' : 
        temp = data.loc[idx]
        temp['state'] = '매수'
        simulation_df = simulation_df.append(temp)
        print('시가에 매수 --->', end='\t')
        continue
            
    # 목표 가격 (7% 수익) 도달시 매도
    if data.loc[idx]['high'] > 1.07 * simulation_df.loc[date + '_0905']['open'] : 
        temp = data.loc[idx]
        temp['state'] = '매도'
        simulation_df = simulation_df.append(temp)
        print('목표가격에 도달하여 종가로 매도 --- >', end='\t')
        break
        
    # 매입가 대비 3% 이상 손해가 난 경우 손절
    temp = data.loc[idx]
    if temp['close'] < 0.97 * simulation_df.loc[date + '_0905']['open'] :
        temp['state'] = '매도'
        simulation_df = simulation_df.append(temp)
        print("매입가 대비 3% 이상 손해가 나서 일괄 매도 --->" ,end='\t')
        break
    
    # 이전 check 단계 대비 종가가 2% 이상 하락 현상이 나타난 경우
    if int(data.loc[idx]['close']) <= 0.98 * int(simulation_df[-1:][['close']].values[0][0]) : 
        temp = data.loc[idx]
        temp['state'] = '매도'
        simulation_df = simulation_df.append(temp)
        print("직전 time point 대비 종가가 2% 하락하여 일괄 매도 --->" ,end='\t')
        break
    
    # 15:00의 종가가 매입가와 오차범위(-0.5 ~ 0.5%) 이내일 때
    if idx == date + '_1500' : 
        temp = data.loc[idx]
        if temp['high'] - simulation_df.loc[date + '_0905']['open'] <\
                                          0.005 * simulation_df.loc[date + '_0905']['open']  :
            temp['state'] = '매도'
            simulation_df = simulation_df.append(temp)
            print("15:00 까지 매입가 대비 움직임이 없어서 일괄 매도 --->", end='\t')
            break
        else : 
            temp['state'] = '보유'
            simulation_df = simulation_df.append(temp)
    
    # 15:20 종가로 일괄 매도
    if idx == date + '_1520' : 
        temp = data.loc[idx]
        temp['state'] = '매도'
        simulation_df = simulation_df.append(temp)
        print('15:20 도달하여 종가로 일괄 매도 --->', end='\t')
        break
    
    # 손절 요건이 충족되지 않을 경우 계속 보유
    temp = data.loc[idx]
    temp['state'] = '보유'
    simulation_df = simulation_df.append(temp)
    
# 시뮬레이션 종료
print('시뮬레이션 종료!')
    
# 피쳐 순서
simulation_df = simulation_df[['open', 'high', 'low', 'close', 'volume', 'state']]

SyntaxError: invalid syntax (<ipython-input-3-be87b2d1d653>, line 57)

#### 05. 시뮬레이션 결과

In [ ]:
simulation_df

#### 06. 수익률

In [ ]:
buy = simulation_df.loc[date + '_0905']['open']
cell = int(simulation_df[-1:]['high'])
rate = (100 * (cell - buy)) / buy
print('종목 코드 : {},     매수가 : {},     매도가 : {},     수익률 : {:.2f}%' .format(code, buy, cell, rate))